# Project 3: LangChain Function/Tool Calling Calculator
In this Project, we will create a simple LangChain Colab Notebook that uses the Google Gemini Flash model to answer user calculation questions. This example below is provided to help you get started assumes you have access to the Gemini API and a basic Python environment. However, we are required to develop and submit our project using Google Colab.

In [1]:
!pip install python-dotenv -q
!pip install langchain-community -U -q
!pip install google-generativeai -U -q # Update `google-generativeai` package
!pip install langchain -U -q
!pip install langchain_google_genai -U -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00


### Step 1: Import API keys  in Colab from secrets


In [11]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

# Step 1: Retrieve the API key from Colab's user data
api_key = userdata.get('GOOGLE_API_KEY')

#### Step 2: Define the Calculator Tool
We define a Calculator class that can evaluate arithmetic expressions. The calculate method uses Python's eval function to compute the result safely.

In [4]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

#### Step 3: Create the Tool Wrapper for LangChain
We integrate the calculator with LangChain as a tool using the Tool class.

In [5]:
from langchain.tools import tool

# Define the tool using a decorator
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

#### Step 4: Set Up the Google Gemini Flash Model
We initialize the Google Gemini Flash model with the API key and define the tools to be used by the agent.

In [13]:
import os
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains.conversation.memory import ConversationBufferMemory

# Initialize the Google Gemini Flash model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model name, e.g., 'gemini-1.5-flash'
    api_key=userdata.get('GOOGLE_API_KEY'),
    temperature=0.7
)
print("Google Gemini Flash model initialized.")


Google Gemini Flash model initialized.


In [15]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool


# Create a list of tools
tools = [calculator]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=gemini_model,
    agent_type="openai-function-calling"
)

# Example usage
response = agent.invoke("What is 12 * 15?")
print(response)



{'input': 'What is 12 * 15?', 'output': '180'}


### Step 5: Build the Conversational Chain
We set up a conversational chain with memory to maintain context across multiple queries.


In [16]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Set up memory to maintain conversation context
memory = ConversationBufferMemory()

# Build the conversational chain
chain = ConversationChain(
    llm=gemini_model,
    memory=memory,
    verbose=True
)

# Example usage
response = chain.invoke("What is 12 * 15?")
print(response)


<ipython-input-16-0ebed092f483>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-16-0ebed092f483>:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 12 * 15?
AI:

> Finished chain.
{'input': 'What is 12 * 15?', 'history': '', 'response': '12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I could have done it in a few ways:  I could have broken it down into (10 * 15) + (2 * 15) = 150 + 30 = 180. Or I could have used the standard long multiplication algorithm.  My internal processes allow me to perform these calculations very quickly.\n'}


#### Step 6: Test the Calculator Tool
We test the calculator tool by processing a series of queries and printing the responses.

In [17]:
# User query
query = "What is 15 divided by 3?"

# Get response from the system
response = chain.invoke(query)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I could have done it in a few ways:  I could have broken it down into (10 * 15) + (2 * 15) = 150 + 30 = 180. Or I could have used the standard long multiplication algorithm.  My internal processes allow me to perform these calculations very quickly.

Human: What is 15 divided by 3?
AI:

> Finished chain.
{'input': 'What is 15 divided by 3?', 'history': 'Human: What is 12 * 15?\nAI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I could have done it in a few ways:  I could have broken it do

In [18]:

queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.invoke(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I could have done it in a few ways:  I could have broken it down into (10 * 15) + (2 * 15) = 150 + 30 = 180. Or I could have used the standard long multiplication algorithm.  My internal processes allow me to perform these calculations very quickly.

Human: What is 15 divided by 3?
AI: 15 divided by 3 is 5.  This is because 3 goes into 15 exactly five times (3 * 5 = 15).  I can perform this division using several algorithms, including long division, but in this simple case, the answer is readily apparent fro

--------------------